In [3]:
import json
import os
from monai.apps.auto3dseg import (
    AlgoEnsembleBestN,
    AlgoEnsembleBuilder,
    import_bundle_algo_history,
)
from monai.utils.enums import AlgoKeys
import optparse
import shutil
from mri_data.file_manager import scan_3Tpioneer_bids
from mri_data.utils import dice_score
from pathlib import Path
import re
import nibabel as nib
import numpy as np

In [4]:
curr_dir = os.getcwd()
dataroot = Path("/mnt/h/3Tpioneer_bids")
work_dir = Path("/home/srs-9/Projects/ms_mri/training_work_dirs/cp_work_dir_pituitary1")  # the algorithm working directory generated by AlgoGen/BundleGen
ensemble_out_dir = work_dir / "ensemble_output" / "3Tpioneer_bids"
save_dir = Path("/mnt/h/3Tpioneer_bids_predictions")

In [5]:
for sub_dir in ensemble_out_dir.glob("sub*"):
    subid = re.match(r"(sub-ms\d{4})", sub_dir.name)[1]
    ses_dirs = [Path(item.path) for item in os.scandir(sub_dir) if "ses" in item.name]

    for ses_dir in ses_dirs:
        sesid = re.match(r"(ses-\d+)", ses_dir.name)[1]
        orig_label = dataroot / subid / sesid / "pituitary.nii.gz"
        pred_label = ses_dir / "t1_ensemble.nii.gz"

        seg1 = nib.load(orig_label).get_fdata()
        seg2 = nib.load(pred_label).get_fdata()

        # Calculate Dice score
        score = dice_score(seg1, seg2)
        print(f'Dice Score: {score}')

Dice Score: 0.8498603351955307
Dice Score: 0.8340141106572596
Dice Score: 0.7672842295526434
Dice Score: 0.8453934361407671
Dice Score: 0.7660343270099368
Dice Score: 0.8885424785658612
Dice Score: 0.7908337396392003
Dice Score: 0.8004330566582462
Dice Score: 0.9001782531194296
Dice Score: 0.776134625191229
Dice Score: 0.8176100628930818
Dice Score: 0.803123983078425
Dice Score: 0.7830775611779345


In [8]:
dataset = scan_3Tpioneer_bids(dataroot,"flair.nii.gz")

In [9]:
# infile = dataset.find_scan(subid="ms1001", ses="20180323").image
infile = dataset[0].image_path
infile.relative_to(dataroot)

PosixPath('sub-ms1001/ses-20170215/flair.nii.gz')

In [11]:
images = []
for i in range(78,80):
    infile: Path = dataset[i].image_path
    images.append({"image": str(infile.relative_to(dataroot))})

In [12]:
# datalist = {
#     "testing": [{"image": infile.name}]
# }

datalist = {
    "testing": images
}

datalist_file = "datalist.json"
with open(datalist_file, "w") as f:
    json.dump(datalist, f)


task = {
  "name": "infer_pit",
  "task": "segmentation",
  "modality": "MRI",
  "datalist": "datalist.json",
  "dataroot": str(dataroot),
}

task_file = os.path.join(curr_dir, "task.json")
with open(task_file, "w") as f:
    json.dump(task, f)


In [13]:
work_dir = "/home/srs-9/Projects/ms_mri/training_work_dirs/cp_work_dir_pituitary1"  # the algorithm working directory generated by AlgoGen/BundleGen
input_cfg = "task.json"  # path to the task input YAML file created by the users

history = import_bundle_algo_history(work_dir, only_trained=True)


In [14]:
## model ensemble
n_best = 5
builder = AlgoEnsembleBuilder(history, input_cfg)
builder.set_ensemble_method(AlgoEnsembleBestN(n_best=n_best))
ensemble = builder.get_ensemble()
save_params = {
    "_target_": "SaveImage", 
    "output_dir": save_dir, 
    "data_root_dir": dataroot, 
    "output_postfix": "pituitary_pred", 
    "separate_folder": False}
                               
pred = ensemble(pred_param = {"image_save_func": save_params})

Ensembling (rank 0)...:   0%|          | 0/2 [00:00<?, ?it/s]monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().
You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `we

KeyboardInterrupt: 